<a href="https://colab.research.google.com/github/michaelherediaperez/jnramirezg/mecanica_de_solidos/blob/main/codigo/11-circulo_mohr_2d_interactivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 11. Círculo de Mohr 2D interactivo

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Michael Heredia Pérez, <mherediap@unal.edu.co> | 2022-07-15 | Código base círculo de Mohr|
|Juan Nicolás Ramírez, <jnramirezg@unal.edu.co> | 2022-09-25 | Agregando sólido e interactivo|



In [1]:
from numpy import *
from numpy import linalg as LA
from matplotlib.pyplot import *
from matplotlib.patches import Polygon
from ipywidgets import interact 

In [2]:
def circulo_mohr(sx, sy, txy):
    
    sigma = array([[sx,  txy],   # Matriz de tensiones.
                   [txy,  sy]])

    valp, vecp = LA.eigh(sigma)          # Cálculo de valores y vectores propios.
    s2, s1 = valp                       # Esfuerzos principales.
    ng1, ng2  = vecp[:, 0], vecp[:, 1]  # Vects. unitarios en las dirs. 1 y 2.
    
    tt = linspace(0, 2*pi, 100)  # Álgulo tetha evaluado de 0° a 180°, en 100 puntos.
    ssn_t = (sx+sy)/2 + (sx-sy)/2*cos(2*tt) + txy*sin(2*tt)  # componente normal de la circunferencia.
    ttn_t = txy*cos(2*tt) - (sx-sy)/2*sin(2*tt)              # componente tangencial del circunferencia.
    
    tmax = R = (((sx-sy)/2)**2 + txy**2)**0.5      # Cálculo el esfuerzo cortante máximo, radio.
    C = (s1+s2)/2  # Centro del círculo en x.                  
    rcParams.update({'font.size': 18})  # Defino tamaño de letra.       
    fig, ax = subplots(figsize=(37*2/3, 18*2/3))  # Inicio el lienzo y defino tamaño.
    plot([-15, 24], [ 0, 0], "-k")  # Eje x.
    plot([  0,  0], [-9, 9], "-k")  # Eje y.

    plot((sx+sy)/2, 0, "*r")  # Dibujo el centro de la circunferencia O.
    plot(ssn_t, ttn_t, "-r")  # Dibujo la circunferencia.
    plot(sx, 0, "*b", label=r"$\sigma_x = $" + f"{sx} Pa")  # Esfuero σx.
    plot(sy, 0, "*b", label=r"$\sigma_y = $" + f"{sy} Pa")  # Esfuero σy.
    plot(s1, 0, "*k", label=r"$\sigma_1 = $" + f"{round(s1,2)} Pa")  # Esfuero σ1.
    plot(s2, 0, "*k", label=r"$\sigma_2 = $" + f"{round(s2,2)} Pa")  # Esfuero σ2.
    plot(s2+R,  tmax, "*k", label=r"$\tau_{max} = $" + f"{round( tmax,2)} Pa")  # Esfuero τmax.
    plot(s2+R, -tmax, "*k", label=r"$\tau_{min} = $" + f"{round(-tmax,2)} Pa")  # Esfuero τmin.
    
    plot([sy, sx], [-txy,  txy], "--g")  # Recta que pasa por los puntos C y A.
    plot([sx, sx], [   0,  txy], "--b")  # Línea de proyección de τxy.
    plot([sy, sy], [   0, -txy], "--b")  # Línea de proyección de -τxy.
    plot([0,  sx], [ txy,  txy], "--b")
    plot([0,  sy], [-txy, -txy], "--b")
    plot(sy, -txy, "*b")  # Punto C.
    plot(sx,  txy, "*b")  # Punto A.
    
    text(0,  txy+0.2, r"$\tau_{xy}$", color="b")
    text(0, -txy-0.6, r"$\tau_{xy}$", color="b")
    text(sx+0.2,  txy+0.2, f"({sx}, { txy})", color="b", fontsize=12, bbox=dict(boxstyle="round", color="white"))  
    text(sy+0.2, -txy+0.2, f"({sy}, {-txy})", color="b", fontsize=12, bbox=dict(boxstyle="round", color="white"))  
    text(sx+0.2, 0.2, r"$\sigma_x$", color="b")
    text(sy-0.6, 0.2, r"$\sigma_y$", color="b") 
    text(s1+0.2, -0.4, r"$\sigma_1$")
    text(s2-0.8, -0.4, r"$\sigma_2$")
    text(s2+R,  tmax+0.2, r"$\tau_{max}$")
    text(s2+R, -tmax-0.4, r"$\tau_{min}$")
    
    # Función que dibuja textos y flechas.
    def flecha_texto(xyf, xyt, var, colorf="r", colort="r"):
        xyt = array(xyt)
        if var < 0: xyf1, xyf2 = array(xyf)[1], array(xyf)[0]
        else:       xyf1, xyf2 = array(xyf)[0], array(xyf)[1]
        ax.annotate("", xy=(xyf1), xytext=(xyf2),
                    arrowprops=dict(arrowstyle="->", color=colorf))
        text(xyt[0], xyt[1], abs(round(var, 2)), color=colort)

    # Funcióm para ajuste numérico del gráfico.
    ajuste     = lambda n,dx,dy:array([ones(n)*dx, ones(n)*dy]).T
    # Fun. para dibujar lineas punteadas negras.
    dibujo_eje = lambda x, y: plot(x, y, "--k", linewidth=1)
    # Función para dibujar un rectángulo.
    rectang = lambda xy: ax.add_patch(Polygon( xy, alpha=1, facecolor='aliceblue',
                                              edgecolor='b', linewidth=2))
    txt_vec_n = lambda n: f"[{round(-n[0],2)}, {round(-n[1],2)}"+r"$]^{T}$"
    
    t1 = arctan2(ng1[1], ng1[0])  # Ángulo dir. principal 1.
    t2 = arctan2(ng2[1], ng2[0])  # Ángulo dir. principal 2.
    
    if t1 < 0: t1 += pi
    if t2 < 0: t2 += pi

    ta = linspace(0, t1)                     # Puntos evaluación de los ángulos.
    tb = linspace(0, t2)
    if t1 < t2:
        tac = linspace(0, 2*t1)
        tbc = linspace(-pi, 2*t1)
    else:
        tac = linspace(-pi, 2*t2)
        tbc = linspace(0, 2*t2)
    
    T = array([[cos(t1), -sin(t1)],   # Matriz trans. para la dir. principal 1.
               [sin(t1),  cos(t1)]])

    a, b = 5.0/2, 3.0/2  # Ancho y alto del rectángulo.
    d = 1.5/2            # Longitud flecha de los esfuerzos.
    ca, cb, cd = a/2, b/2, d/2  # Valores medios ancho, largo y longitud de flecha.

    cent  = array([ca, cb])  # Centro del rectángulo original.
    centP = cent@T.T         # Centro del rectángulo transformado.
    delta   = cent - centP   # Diferencia en la ubicación de los centros.
    dx, dy  = 2.5*a + 3 + delta[0], 2+delta[1]  # Ajuste x y y rectángulo transformado.
    c2 = centP + ajuste(1, dx, dy)[0]     # Centro rectángulo trans. con ajuste.
    vert  = array([[0,0],[0,b],[a,b],[a,0]])  # Vértices rectángulo original.
    vertP = vert@T.T + ajuste(4, dx, dy)      # Vértices rectángulo transformado.

    # Matrices información de la ubicación de las flechas y sus valores.
    A = array([[[ a, cb], [ d+.5,    0], [ .5,  0], [  d-.2,    .2]],   # sx der.
               [[ca,  b], [    0, d+.5], [  0, .5], [    .2,  d-.2]],   # sy sup.
               [[ 0, cb], [-d-.5,    0], [-.5,  0], [ -d+.2,    .2]],   # sx izq.
               [[ca,  0], [    0,-d-.5], [  0,-.5], [    .2, -d+.2]]])  # sy inf.
    
    # Matriz información: ajust posición valor, ajust posición ng, valor, texto ng.
    F = [[[ 0.0, 0.5], [ 1.0, 0.0], s1, r"$\hat{n}_1$=" + txt_vec_n(ng1)],  # ng1(+).
         [[ 0.0, 0.0], [-2.0, 1.0], s2, r"$\hat{n}_2$=" + txt_vec_n(ng2)],  # ng2(+).
         [[-0.5,-0.5], [-0.0, 0.0], s1,                   txt_vec_n(ng1)],  # ng1(-).       
         [[ 0.0,-0.5], [-0.0,-0.0], s2,                   txt_vec_n(ng2)]]  # ng2(-).

    S, U= [sx, sy, sx, sy], [txy, txy, txy, txy]  # Auxiliar de almacenamiento.

    # Gráfica de sólido.
    rcParams.update({'font.size': 12})  # Defino tamaño de letra.

    dibujo_eje([c2[0],         c2[0]+5], [c2[1],           c2[1]])     # Dib. eje x auxiliar.
    dibujo_eje([c2[0], c2[0]+4/2*cos(t1)], [c2[1], c2[1]+4/2*sin(t1)])  # Dib. dir 1.
    dibujo_eje([c2[0], c2[0]+3/2*cos(t2)], [c2[1], c2[1]+3/2*sin(t2)])  # Dib. dir 2.
    plot(c2[0] + 3/2*cos(ta), c2[1] + 3/2*sin(ta),"-m", linewidth=0.8)  # Dib. t1 en sólido.
    plot(c2[0] +   1*cos(tb), c2[1] +   1*sin(tb),"-c", linewidth=0.8)  # Dib. t2 en sólido.
    plot(C + 1.0*cos(tac),  1.0*sin(tac),"-m", linewidth=0.8, label=r"$\theta_1=$"+f"{round(rad2deg(t1),2)}°")  # Dib. t1 en círculo.
    plot(C + 0.5*cos(tbc),  0.5*sin(tbc),"-c", linewidth=0.8, label=r"$\theta_2=$"+f"{round(rad2deg(t2),2)}°")  # Dib. t2 en círculo.
    rectang(vertP)  # Dibujo de rectángulo transformado.

    for i in range(len(A)): # Dibujo de las flechas y los valores.
        xy_f = array([A[i][0]+A[i][1], A[i][0]+A[i][2]*0.5])@T.T + ajuste(2,dx,dy)
        # Coordenadas textos transformadas.
        xy_t = (A[i][0]+A[i][3]+ F[i][0])@T.T + ajuste(1, dx, dy)[0]
        # Flechas de las direcciones principales, sus valores y sus vectores ng.
        flecha_texto(xy_f, xy_t,  F[i][2])
        if i in[0,1]:
        # Textos de los vectores ng.
            text(xy_t[0]+F[i][1][0], xy_t[1]+F[i][1][1], F[i][3], color="k")

    # Impresión del valor de los ángulos t1 y t2.
    nul1, nul2 = 1, 1  # Anulan cálculo con condición, inicialmente no hacen nada.
    if isclose(t1, 0):   nul1 =0  # Si t1 es cero, entonces cero1 anula su texto.
    elif isclose(t2, 0): nul2 =0  # Si t2 es cero, entonces cero1 anula su texto.
    text((c2[0]+3/2*cos(t1/2)+.2)*nul1, c2[1]+3/2*sin(t1/2)-.1, r"$\theta_1$"*nul1, fontsize=9, bbox=dict(boxstyle="round", color="white"))
    text((c2[0]+0.8*cos(2*t2/3)+.2)*nul2, c2[1]+0.8*sin(2*t2/3)-.1, r"$\theta_2$"*nul2, fontsize=9, bbox=dict(boxstyle="round", color="white"))
    text((C    +1.0*cos(4*t1/3))*nul1, 1.0*sin(4*t1/3), r"2$\theta_1$"*nul1, fontsize=9)
    text((C    +0.5*cos(-t2))*nul1, 0.5*sin(-t2), r"2$\theta_2$"*nul1, fontsize=9)
    
    xlim(-15, 22)
    ylim(-9, 9)
    xlabel("Esfuerzo normal " + r"$\sigma_n$")  # Nombre del eje x.
    ylabel("Esfuerzo cortante " + r"$\tau_n$")  # Nombre del eje y.
    grid(b=True, which='both', color='k', linestyle='-', alpha=0.1)
    legend()
    show()               # Mostrar gráfica.

In [3]:
interact(circulo_mohr, 
         sx  = (-8, 6, 0.5), 
         sy  = (-3, 7, 0.5), 
         txy = (-10, 4, 0.5))
None

interactive(children=(FloatSlider(value=-1.0, description='sx', max=6.0, min=-8.0, step=0.5), FloatSlider(valu…